<a href="https://colab.research.google.com/github/IlyaKuprik/skolkovo-hack/blob/main/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Process

## загрузка


In [ ]:
{"Самоотказ РЦ": 1, "Не отвечает": 0.1, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 0.7, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3, 'Вышел на линию': 0.9,
  "Принят на работу": 0.95, "Оформлен": 0.9, "Неактуально": 0, "Отказ компании": 0, "Перезаписался": 0.4, "Отказался/Нашел работу": 0.3,
  "Заключил договор": 0.8, "Отправлен на СБ": 0.85, "Интервью с HR": 0.85, "Самоотказ МП": 0.5, "Недозвон": 0.2, "Отказ по итогам скрининга резюме": 0,
  "Не прошёл СБ": 0.5, "Отказ HR": 0, "Резерв РЦ": 0.2, "Не пришел (повторно)": 0.6, "Интервью": 0.8, "Отказ ВВСС (другое)": 0.2, "Отказ СБ": 0.65}

In [2]:
data_candidates_work_places = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_work_places.csv', sep=';',
                                names = ['CandidateId', 'Position','FromYear', 'FromMonth', 'ToYear', 'ToMonth'])

In [3]:
data_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/data_jobs.csv', sep=';',
                                names = ['JobId', 'Status','Name', 'Region', 'Description'])

In [4]:
data_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates.csv', sep=';',
                                names = ['CandidateId', 'Position','Sex', 'Citizenship', 'Age',
                                'Salary', 'Langs', 'DriverLicense', 'Subway', 'Skills', 'Employment', 'Shedule',                                           
                                 'CandidateRegion', 'DateCreated', 'JobId', 'CandidateStatusId', 'Status'])

In [5]:
data_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_education.csv', sep=';',
                                names = ['CandidateId', 'University', 'Faculty', 'GraduateYear'])

## Обработка таблицы

In [6]:
import re
 
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|-|/|;|_|•')
 
def cleanhtml(raw_html):
    if type(raw_html) == type(''):
        return re.sub(CLEANR, '', raw_html)
    return raw_html
 
data_jobs.Description = data_jobs.Description.apply(cleanhtml)

In [7]:
cols_val = {str(i): 'last' for i in data_candidates.columns}

In [8]:
grouped_with_last_status = data_candidates.sort_values(by = 'DateCreated').loc[:,data_candidates.columns]\
.groupby(by = ['CandidateId','JobId'], axis = 0, sort = False, as_index = False).agg( cols_val)

### Штрафы

In [9]:
target_map = {"Самоотказ РЦ": 1, "Не отвечает": 0.1, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 0.7, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3, 'Вышел на линию': 0.9,
  "Принят на работу": 0.95, "Оформлен": 0.9, "Неактуально": 0, "Отказ компании": 0, "Перезаписался": 0.4, "Отказался/Нашел работу": 0.3,
  "Заключил договор": 0.8, "Отправлен на СБ": 0.85, "Интервью с HR": 0.85, "Самоотказ МП": 0.5, "Недозвон": 0.2, "Отказ по итогам скрининга резюме": 0,
  "Не прошёл СБ": 0.5, "Отказ HR": 0, "Резерв РЦ": 0.2, "Не пришел (повторно)": 0.6, "Интервью": 0.8, "Отказ ВВСС (другое)": 0.2, "Отказ СБ": 0.65}

In [10]:
targets = grouped_with_last_status['Status'].apply(lambda x: target_map[x] if x in target_map else 1/2) ## Заполнили NA
grouped_with_ids_target = grouped_with_last_status.loc[:, ['CandidateId', 'JobId',]]
grouped_with_ids_target['target'] = targets

In [12]:
#Инициализация штрафов
number_fields = ['Age', 'Salary']
string_fields = ['Position', 'Citizenship', 'Langs',\
                  'Skills', 'Employment', 'Shedule', 'CandidateRegion']
fines_vector = [1/18]*9 

In [13]:

mask_df = grouped_with_last_status.copy()
mask_df[number_fields] = grouped_with_last_status[number_fields].apply(lambda x: x == 0)
mask_df[string_fields] = grouped_with_last_status[string_fields].apply(lambda x: x.isna())

total_fine = mask_df[number_fields + string_fields].apply(lambda x: 1 - (np.array(x) * np.array(fines_vector)).sum(), axis = 1)
total_target = targets*total_fine
grouped_with_ids_target['target'] = total_target

## Driver license

In [14]:
def del_sep(text):
    if type(text) == type(''):
        return ''.join(text.split(','))
    return text

data_candidates.DriverLicense = data_candidates.DriverLicense.apply(del_sep)
data_candidates.DriverLicense.isna().sum()

93730

In [96]:

# for i, row in data_candidates.iterrows():
#     if type(row.DriverLicense) != type('') and type(row.Position) == type(''):
#         if np.isnan(row.DriverLicense):
#             Flag = False
#             if 'водитель' in row.Position.lower():
#                 if not Flag:
#                     data_candidates.iloc[i, 7] = 'B'
#                     Flag = True
#             if 'грузового' in row.Position.lower() or 'грузовой' in row.Position.lower():
#                 if not Flag:
#                     data_candidates.iloc[i, 7] = 'C'
#                     Flag = True
#                 else:
#                     data_candidates.iloc[i, 7] += 'C'
#             if Flag == False:
#                 data_candidates.iloc[i, 7] = 'N'
# data_candidates.DriverLicense.isna().sum()

47288

## citizenship

In [18]:
data_candidates.Citizenship

0         Россия
1         Россия
2         Россия
3         Россия
4         Россия
           ...  
141884    Россия
141885    Россия
141886    Россия
141887    Россия
141888    Россия
Name: Citizenship, Length: 141889, dtype: object

In [ ]:
data_candidates.Citizenship.apply(lambda x: 'Россия' if 'рос' in x.lower() or 'рф' in x.lower() or 'rus' in x.lower()\
                                  else 'Киргизия' if x.lower().startswith('к')\
                                  else 'Армения' if x.lower().startswith('а')\
                                  else 'Узбекистан' if x.lower().startswith('у')
                                  else 'Таджикистан')

In [98]:
for i, row in data_candidates.iterrows():
    if type(row.Citizenship) == type(''):
        if 'рос' in row.Citizenship.lower() or 'рф' in row.Citizenship.lower() or 'rus' in row.Citizenship.lower():
            data_candidates.iloc[i, 3] = 'Россия'
        if row.Citizenship.lower().startswith('к'):
            data_candidates.iloc[i, 3] = 'Киргизия'
        if row.Citizenship.lower().startswith('а'):
            data_candidates.iloc[i, 3] = 'Армения'
        if row.Citizenship.lower().startswith('у'):
            data_candidates.iloc[i, 3] = 'Узбекистан'
        if row.Citizenship.lower().startswith('т'):
            data_candidates.iloc[i, 3] = 'Таджикистан'

## Skills

In [15]:
def del_stick(text):
    if type(text) == type(''):
        return ''.join(text.split('||'))
    return text


data_candidates.Skills = data_candidates.Skills.apply(del_stick)

### Обработка текста

In [16]:
grouped_table_candidates_features = grouped_with_last_status.drop(columns = ['Status', 'CandidateStatusId'])

In [ ]:
grouped_table_candidates_features

In [18]:
new_df = grouped_table_candidates_features\
    .merge(data_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')
new_df.index = [new_df.CandidateId, new_df.JobId]
df = new_df.drop(columns=['JobId', 'CandidateId', 'Salary', 'Langs', 'Subway',
                          'Age', 'GraduateYear', 'DateCreated'])
df.Sex = df.Sex\
    .where(~(df.Sex == 2), other='мужчина')\
    .where(~(df.Sex == 1), other='женщина')\
    .where(~(df.Sex == 0), other='неопределен')
df = df.fillna('NaN').astype(str).add('. ').sum(axis=1).reset_index()
 
status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = data_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
job_user_tokens1 = new_data_jobs\
    .drop(columns=['Status', 'JobId'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()
 
job_user_tokens2 = df
job_user_tokens_with_id = job_user_tokens2\
    .merge(job_user_tokens1, on='JobId')
 

In [19]:
merged_descriptions_targets_ids = job_user_tokens_with_id.merge(grouped_with_ids_target, on = ['JobId', 'CandidateId'], how = 'inner')
merged_descriptions_targets_ids.columns = ["CandidateId", "JobId", "Candidate_descr", "Jobs_descr", "target"]
merged_descriptions_targets_ids = merged_descriptions_targets_ids[~(merged_descriptions_targets_ids.target.isna())]

In [30]:
# merged_descriptions_targets_ids = job_user_tokens_with_id.merge(grouped_with_ids_target, on = ['JobId', 'CandidateId'], how = 'inner')
# merged_descriptions_targets_ids.columns = ["CandidateId", "JobId", "Candidate_descr", "Jobs_descr", "target"]
# merged_descriptions_targets_ids.isna().sum()
# merged_descriptions_targets_ids = merged_descriptions_targets_ids.fillna(1/2)

In [20]:
merged_descriptions_targets_ids.drop(columns = ['CandidateId', 'JobId'], inplace = True)

In [21]:
merged_descriptions_targets_ids

,Candidate_descr,Jobs_descr,target
0,Водитель. мужчина. Гражданство. NaN. NaN. Full...,Водитель. Санкт-Петербург. Требования к опыту...,0.388889
1,Водитель. неопределен. NaN. NaN. NaN. Full. Fu...,Водитель. Санкт-Петербург. Требования к опыту...,0.633333
2,Водитель. мужчина. Россия. NaN. NaN. Full. Ful...,Водитель. Санкт-Петербург. Требования к опыту...,0.388889
3,Водитель. мужчина. Россия. NaN. NaN. Full. Ful...,Водитель. Санкт-Петербург. Требования к опыту...,0.388889
4,Водитель. мужчина. Россия. NaN. NaN. Full. Ful...,Водитель. Санкт-Петербург. Требования к опыту...,0.388889
...,...,...,...
57353,Водитель ричтрака (погрузчика) на складе катег...,Водитель погрузчика / штабелера. Москва. Водит...,0.472222
57354,Водитель погрузчика. мужчина. Россия. B. Дейст...,Водитель погрузчика / штабелера. Москва. Водит...,0.000000
57355,Кладовщик-комплектовщик. мужчина. Россия. BC. ...,Водитель погрузчика / штабелера. Москва. Водит...,0.700000
57356,"Водитель погрузчика, Штабелера. мужчина. Росси...",Водитель погрузчика / штабелера. Москва. Водит...,0.661111


In [22]:
merged_descriptions_targets_ids.to_csv('preprocess_data.csv')

# Обработка теста


In [24]:
test_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates.csv', sep = ';')

In [65]:
test_candidates.head()

,CandidateId,Position,Sex,Citizenship,Age,Salary,Langs,DriverLicense,Subway,Skills,Employment,Schedule,CandidateRegion
0,8458568,специалист отдела парковочного комплекса,2,Россия,50,45000,английский:базовые знания||русский:родной||,NaN,NaN,"Ведение документации ,управление производством...",Part,NaN,Санкт-Петербург
1,8480519,Оператор,2,Россия,50,50000,русский:родной||немецкий:базовые знания||,NaN,Ленинский проспект,Водительское удостоверение категории BCD || п...,Part,Flex,Санкт-Петербург
2,8511395,"Ученик автослесаря , автомеханика .",2,Россия,24,50000,русский:родной||английский:базовые знания||,NaN,NaN,Работоспособность || Навыки коммуникации || Ра...,Part,NaN,Санкт-Петербург
3,8511459,Агент по обработке авиаперевозок,2,Россия,33,60000,русский:родной||,NaN,NaN,способность заниматься несколькими проектами |...,Part,Flex,Москва
4,8530237,Техник-механик,2,Россия,50,45000,русский:родной||английский:свободно владею||,NaN,NaN,Активная позиция || ПК. Excel. Word || Трактор...,Look-out,NaN,Санкт-Петербург


In [25]:
test_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_jobs.csv', sep = ';',\
                        names = ['JobId', 'Status','Name', 'Region', 'Description', 'x', 'y','z'])
test_jobs.drop(columns = ['x', 'y','z'], inplace = True)

In [64]:
test_jobs.head()

,JobId,Status,Name,Region,Description
0,140952,2,Водитель автомобиля большегруза с прицепом,Санкт-Петербург,ОБЯЗАННОСТИ: управление транспортными средств...
1,76137,2,водитель на своем авто,Москва,"Начался сезон отпусков, по этой причине открыв..."
2,119729,4,Водитель,Москва,"Наш клиент, крупный холдинг рассматривает канд..."
3,62241,2,Водитель на своем авто,Москва,Стаж вождения от 3х летОбразование не ниже сре...
4,63795,2,Водитель,Санкт-Петербург,Обязанности:Качественная доставка японской кух...


In [26]:
test_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/test/test_candidates_education.csv', sep=';')

In [71]:
test_candidates_education.head()

,CandidateId,University,Faculty,GraduateYear
0,6896427,ГБПОУ КСУ 32,"Очное , Повар Кондитер",2016
1,7271897,МГТА,"Юриспруденция, Юрист",2015
2,7496524,Московский городской педагогический университе...,"Юриспруденция, Юриспруденция",2018
3,7609878,ТУ №12,автомеханик,2002
4,7637904,Российская Государственная академия физической...,"Физическая культура, Преподаватель физической ...",1995


## Обработка Жопы

In [27]:
import re
 
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|-|/|;|_|•')
 
def cleanhtml(raw_html):
    if type(raw_html) == type(''):
        return re.sub(CLEANR, '', raw_html)
    return raw_html
 
test_jobs.Description = test_jobs.Description.apply(cleanhtml)

In [28]:
status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = test_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
job_user_tokens1 = new_data_jobs\
    .drop(columns=['Status', 'JobId'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()

In [63]:
job_user_tokens1

,JobId,0
0,140952,Водитель автомобиля большегруза с прицепом . С...
1,76137,водитель на своем авто . Москва. Начался сезон...
2,62241,Водитель на своем авто. Москва. Стаж вождения ...
3,63795,Водитель. Санкт-Петербург. Обязанности:Качеств...
4,94661,водитель на личном авто. Москва. ищем курьера ...
5,120695,Водитель-экспедитор 1/ Водитель. Москва. Обяза...
6,157891,водитель курьер на авто компании. Москва. води...
7,165605,Менеджер по продажам. Санкт-Петербург. Обязанн...
8,241719,Водитель-экспедитор. Москва. ЗАДАЧИ: Доставка...
9,119729,"Водитель. Москва. Наш клиент, крупный холдинг ..."


## Обработка кандидата

In [29]:
def del_sep(text):
    if type(text) == type(''):
        return ''.join(text.split(','))
    return text

test_candidates.DriverLicense = test_candidates.DriverLicense.apply(del_sep)
test_candidates.DriverLicense.isna().sum()

90

In [30]:
def del_stick(text):
    if type(text) == type(''):
        return ''.join(text.split('||'))
    return text


test_candidates.Skills = test_candidates.Skills.apply(del_stick)

In [69]:
test_candidates

,CandidateId,Position,Sex,Citizenship,Age,Salary,Langs,DriverLicense,Subway,Skills,Employment,Schedule,CandidateRegion
0,8458568,специалист отдела парковочного комплекса,2,Россия,50,45000,английский:базовые знания||русский:родной||,NaN,NaN,"Ведение документации ,управление производством...",Part,NaN,Санкт-Петербург
1,8480519,Оператор,2,Россия,50,50000,русский:родной||немецкий:базовые знания||,NaN,Ленинский проспект,Водительское удостоверение категории BCD про...,Part,Flex,Санкт-Петербург
2,8511395,"Ученик автослесаря , автомеханика .",2,Россия,24,50000,русский:родной||английский:базовые знания||,NaN,NaN,Работоспособность Навыки коммуникации Работа...,Part,NaN,Санкт-Петербург
3,8511459,Агент по обработке авиаперевозок,2,Россия,33,60000,русский:родной||,NaN,NaN,способность заниматься несколькими проектами ...,Part,Flex,Москва
4,8530237,Техник-механик,2,Россия,50,45000,русский:родной||английский:свободно владею||,NaN,NaN,Активная позиция ПК. Excel. Word Тракторные ...,Look-out,NaN,Санкт-Петербург
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,339,"Менеджер по продажам, ведущий менеджер",1,Россия,0,45000,Русский : родной||Английский : базовые знания||,NaN,м. Петроградская,Знание техники продаж. Умение вести переговоры...,Full,Full,Санкт-Петербург
196,585,"Менеджер по продажам, закупкам",2,Россия,0,30000,английский:базовый||,B,Проспект&nbsp;Ветеранов,NaN,Full,Full,Санкт-Петербург
197,616,Менеджер по продажам,2,Россия,30,40000,русский:базовые знания||английский:могу проход...,NaN,NaN,NaN,Full,Full,Санкт-Петербург
198,622,Менеджер по продажам,2,Россия,30,40000,русский:базовые знания||английский:могу проход...,NaN,м. Новочеркасская,NaN,Full,Full,Санкт-Петербург


In [31]:
test_candidates.head()

,CandidateId,Position,Sex,Citizenship,Age,Salary,Langs,DriverLicense,Subway,Skills,Employment,Schedule,CandidateRegion
0,8458568,специалист отдела парковочного комплекса,2,Россия,50,45000,английский:базовые знания||русский:родной||,NaN,NaN,"Ведение документации ,управление производством...",Part,NaN,Санкт-Петербург
1,8480519,Оператор,2,Россия,50,50000,русский:родной||немецкий:базовые знания||,NaN,Ленинский проспект,Водительское удостоверение категории BCD про...,Part,Flex,Санкт-Петербург
2,8511395,"Ученик автослесаря , автомеханика .",2,Россия,24,50000,русский:родной||английский:базовые знания||,NaN,NaN,Работоспособность Навыки коммуникации Работа...,Part,NaN,Санкт-Петербург
3,8511459,Агент по обработке авиаперевозок,2,Россия,33,60000,русский:родной||,NaN,NaN,способность заниматься несколькими проектами ...,Part,Flex,Москва
4,8530237,Техник-механик,2,Россия,50,45000,русский:родной||английский:свободно владею||,NaN,NaN,Активная позиция ПК. Excel. Word Тракторные ...,Look-out,NaN,Санкт-Петербург


In [32]:
new_df = test_candidates\
    .merge(test_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')
new_df.index = new_df.CandidateId
df = new_df.drop(columns=['CandidateId', 'Salary', 'Langs', 'Subway',
                          'Age', 'GraduateYear'])
df.Sex = df.Sex\
    .where(~(df.Sex == 2), other='мужчина')\
    .where(~(df.Sex == 1), other='женщина')\
    .where(~(df.Sex == 0), other='неопределен')
df = df.fillna('NaN').astype(str).add('. ').sum(axis=1).reset_index()

job_user_tokens2 = df

In [45]:
job_user_tokens2 = job_user_tokens2.rename(columns = {0: 'Candidate_descr'})
job_user_tokens1 = job_user_tokens1.rename(columns = {0: 'Jobs_descr'})
job_user_tokens2.drop(columns = ['CandidateId'], inplace = True)
job_user_tokens1.drop(columns = ['JobId'], inplace = True)



In [46]:
job_user_tokens1

,Jobs_descr
0,Водитель автомобиля большегруза с прицепом . С...
1,водитель на своем авто . Москва. Начался сезон...
2,Водитель на своем авто. Москва. Стаж вождения ...
3,Водитель. Санкт-Петербург. Обязанности:Качеств...
4,водитель на личном авто. Москва. ищем курьера ...
5,Водитель-экспедитор 1/ Водитель. Москва. Обяза...
6,водитель курьер на авто компании. Москва. води...
7,Менеджер по продажам. Санкт-Петербург. Обязанн...
8,Водитель-экспедитор. Москва. ЗАДАЧИ: Доставка...
9,"Водитель. Москва. Наш клиент, крупный холдинг ..."


In [47]:
job_user_tokens2

,Candidate_descr
0,специалист отдела парковочного комплекса. мужч...
1,Оператор. мужчина. Россия. NaN. Водительское ...
2,"Ученик автослесаря , автомеханика .. мужчина. ..."
3,Агент по обработке авиаперевозок. мужчина. Рос...
4,Техник-механик. мужчина. Россия. NaN. Активная...
...,...
195,"Менеджер по продажам, ведущий менеджер. женщин..."
196,"Менеджер по продажам, закупкам. мужчина. Росси..."
197,Менеджер по продажам. мужчина. Россия. NaN. N...
198,Менеджер по продажам. мужчина. Россия. NaN. N...


In [49]:
test_data_merged = job_user_tokens2.merge(job_user_tokens1, how = 'cross')

In [50]:
test_data_merged.to_csv('test_data_merged.csv')